### Test functions for DRP_nb module

In [150]:
from importlib import reload
import DRP_nb
import os

In [137]:
os.path.dirname(os.getcwd())

'/data/home/wpw035'

In [177]:
import DRP_nb.data_imports
#from DRP_nb import data_imports

In [178]:
reload(DRP_nb.data_imports)

<module 'DRP_nb.data_imports' from '/data/home/wpw035/Codebase/DRP_nb/data_imports.py'>

In [180]:
#input phos prot rna (ppr) data
inp_ppr = DRP_nb.data_imports.DrpInputData(omic_types=['phos'], drug_rep='smile')

Num smiles that will be shortened 0
Mean number of padding characters needed 61.13732394366197


In [174]:
#take out disjoint cls
inp_ppr.remove_disjoint()

In [185]:
inp_ppr.marker_drugs

,(5Z)-7-Oxozeaenol,5-Fluorouracil,A-443654,A-770041,A-83-01,ACY-1215,AGI-6780,AICA Ribonucleotide,AKT inhibitor VIII,AR-42,...,XMD8-92,Y-39983,YK-4-279,YM201636,Z-LLNle-CHO,ZG-10,ZM447439,ZSTK474,Zibotentan,kb NB 142-70
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
280,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
281,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
282,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
283,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [184]:
inp_ppr.dths

{'(5Z)-7-Oxozeaenol': array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]]),
 '5-Fluorouracil': array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]]),
 'A-443654': array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]]),
 'A-770041': array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., 

In [161]:
inp_ppr.phos

col.name,KDM1A(S45);,KDM1A(T19);,ZFP91(S69);,INCENP(T150);,INCENP(T153);,INCENP(T153);INCENP(T135);,INCENP(M136);INCENP(T145);INCENP(S148);,EIF3J(S11);EIF3J(S13);,POLE4(S9);,SAMD1(T157);,...,CTDP1(T340);,EPS8L2(T572);,BEGAIN(S455);,HNRNPL(S52);,GMDS(T327);,BOD1L1(S482);BOD1L1(S484);,ARHGEF35(T193);,ARHGEF5(S184);,PSMB2(T148);,HSP90AA1(S476);


In [134]:
inp_ppr.phos

col.name,KDM1A(S45);,KDM1A(T19);,ZFP91(S69);,INCENP(T150);,INCENP(T153);,INCENP(T153);INCENP(T135);,INCENP(M136);INCENP(T145);INCENP(S148);,EIF3J(S11);EIF3J(S13);,POLE4(S9);,SAMD1(T157);,...,CTDP1(T340);,EPS8L2(T572);,BEGAIN(S455);,HNRNPL(S52);,GMDS(T327);,BOD1L1(S482);BOD1L1(S484);,ARHGEF35(T193);,ARHGEF5(S184);,PSMB2(T148);,HSP90AA1(S476);
CMK,-9.751659,-9.751659,6.298313,3.897353,6.801432,6.270551,0.925999,2.077243,-9.751659,8.603255,...,-9.751659,-8.751659,4.683472,4.285402,-9.751659,-0.388355,-9.751659,-9.751659,0.575312,1.627607
COLO-680N,-9.158429,-9.158429,4.479295,5.005041,6.620465,5.830401,0.765535,0.487229,-0.349235,7.013462,...,-9.158429,7.549977,5.488547,4.649615,-9.158429,-9.158429,0.214125,1.263034,-2.403542,3.378512
GDM-1,-9.522178,-9.522178,2.807355,4.263786,6.313452,5.388925,0.356144,1.062812,3.184280,8.247928,...,-9.522178,-8.522178,7.215484,4.935460,-0.954557,-0.948976,-9.522178,-9.522178,1.952334,3.320485
HEL,-11.117787,0.641546,3.539779,3.805292,6.411765,5.724105,-11.117787,1.891419,-11.117787,8.733354,...,-11.117787,0.333939,3.135535,4.209453,-1.577767,-11.117787,-11.117787,-11.117787,0.389567,3.279471
HL-60,-11.200250,-11.200250,5.420314,3.594549,4.385697,4.070426,-11.200250,4.852998,-11.200250,5.044394,...,3.584963,-10.200250,7.209458,4.678072,-11.200250,-11.200250,-11.200250,-11.200250,-2.011588,2.536053
JHH-2,-10.000831,-10.000831,2.356419,2.653284,4.659352,3.273661,-10.000831,-9.000831,-10.000831,6.195348,...,-10.000831,7.845490,5.852249,2.985500,4.270529,-10.000831,4.153805,3.292782,-10.000831,2.545968
JHH-4,1.790772,-9.532825,4.705917,4.863014,6.920043,6.361926,1.773996,0.743300,-9.532825,6.137504,...,-9.532825,5.529509,6.375853,3.877744,1.713696,-9.532825,2.839960,0.773996,-9.532825,3.336283
KASUMI-1,-10.480357,-10.480357,5.850999,3.267641,5.040923,4.307480,-10.480357,1.989139,-10.480357,8.485829,...,2.427606,1.914832,-1.416192,2.998196,-10.480357,2.292782,-10.480357,-10.480357,-10.480357,-10.480357
KG-1,-10.582840,-10.582840,6.659542,1.531395,4.475776,3.744231,-10.582840,2.742006,-10.582840,7.156842,...,-10.582840,-1.130833,6.456160,3.523562,4.201634,0.137504,-10.582840,-10.582840,2.223423,2.223423
KMOE-2,4.572890,1.778209,5.472455,3.517276,5.602310,4.672462,-10.602894,3.121015,0.871844,7.660353,...,0.887525,1.541338,4.222650,3.459432,-10.602894,-10.602894,-10.602894,-10.602894,-10.602894,2.639232


In [6]:
prot, rna, ic50_df1, one_hot_cls, one_hot_drugs = DRP_nb.data_imports.read_inputs_phos_ect()
#prot, rna, ic50_df1, one_hot_cls, one_hot_drugs = data_imports.read_inputs_rna_prot()

/data/home/wpw035/.conda/envs/tfGPUforge/lib/python3.9/site-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


% of missing prot values 0.386335609896865
num non overlapping prot and target cls: 10
num non overlapping cls: 930


ValueError: too many values to unpack (expected 5)

In [11]:
if not []:
    print(1)

1


In [13]:
print('123423352\
1')

1234233521


In [102]:
a = 3
d = {'a': a, 'b': 2}
for v in d.values():
    a = 5

In [145]:
d.values()

TypeError: 'dict_values' object is not subscriptable

In [146]:
[1, 2, 3][1 : ]

[2, 3]